In [16]:
import numpy as np
import matplotlib.pyplot as plt

import chainer
import chainer.optimizers as Opt
import chainer.functions as F
import chainer.links as L
import chainer.datasets as ds
import chainer.dataset.convert as con
from chainer.iterators import SerialIterator as siter
from chainer import optimizer_hooks as oph
from chainer import Variable,Chain,config,cuda

from tqdm import tqdm
from IPython import display
import princess as ohm

In [2]:
all_list=[]
ohm.add_labeled_data("princess",0,all_list)
dataset=ds.LabeledImageDataset(all_list)
train=ds.TransformDataset(dataset,ohm.labeled64)
xtrain,_=con.concat_examples(train)
Dtrain,ch,Ny,Nx=xtrain.shape
print(Dtrain,ch,Ny,Nx)

200 3 64 64


In [9]:
class Generator(Chain):
    def __init__(self,zsize,Nx=Nx,Ny=Ny,H1=256*ch,H2=64*ch,H3=16*ch,H4=4*ch):
        layers={}
        layers["l1"]=L.Linear(zsize,Ny//16*Nx//16*H1)
        layers["bnorm1"]=L.BatchNormalization(H1)
        layers["dcnn1"]=ohm.CBR(H1,H2,"up")
        layers["dcnn2"]=ohm.CBR(H2,H3,"up")
        layers["dcnn3"]=ohm.CBR(H3,H4,"up")
        layers["dcnn4"]=ohm.CBR(H4,ch,"up")
        super().__init__(**layers)
    def __call__(self,x):
        h=self.l1(x)
        h=F.dropout(h)
        h=F.relu(h)
        h=h.reshape(len(h),256*ch,4,4)
        h=self.bnorm1(h)
        h=self.dcnn1(h)
        h=self.dcnn2(h)
        h=self.dcnn3(h)
        h=self.dcnn4(h)
        y=F.clip(h,0.0,1.0)
        return y

In [18]:
class Discriminator(Chain):
        def __init__(self,C,ch=ch,H1=64,H2=128,H3=256,H4=512):
            layers={}
            layers["cnn1"]=ohm.CBR(ch,H1,"down",bn=False,act=F.leaky_relu)
            layers["cnn2"]=ohm.CBR(H1,H2,"down",bn=False,act=F.leaky_relu)
            layers["cnn3"]=ohm.CBR(H2,H3,"down",bn=False,act=F.leaky_relu)
            layers["cnn4"]=ohm.CBR(H3,H4,"down",bn=False,act=F.leaky_relu)
            layers["l1"]=L.Linear(None,C)
            super().__init__(**layers)
        def __call__(self,x):
            h=self.cnn1(x)
            h=self.cnn2(h)
            h=self.cnn3(h)
            h=self.cnn4(h)
            h=self.l1(h)
            y=F.dropout(h)
            return y

In [19]:
zsize=2
C=1
gen=Generator(zsize)
dis=Discriminator(C)

In [20]:
optgen=Opt.Adam(alpha=0.0005,beta1=0.5)
optgen.setup(gen)
#optgen.add_hook(oph.GradientClipping(0.1))

optdis=Opt.Adam(alpha=0.0001,beta1=0.5)
optdis.setup(dis)
#optgen.add_hook(oph.GradientClipping(0.1))

#cuda.get_device(0).use()
#gen.to_gpu()
#dis.to_gpu()
train_gen_loss=[]
train_dis_loss1=[]
train_dis_loss2=[]
result=[train_gen_loss,train_dis_loss1,train_dis_loss2]


In [21]:
def learning_GAN(gen,dis,optgen,optdis,data,result,T=1):
    for time in range(T):
        optgen.target.cleargrads()
        ytemp=gen(data[1])
        with chainer.using_config("train",False):
            ytrain_false=dis(ytemp)
        loss_train_gen=0.5*F.mean((ytrain_false-1.0)**2)
        #loss_train_gen=0.5*F.mean(F.softplus(-ytrain_false))
        loss_train_gen.backward()
        optgen.update()

        optdis.target.cleargrads()
        ytrain_false=dis(ytemp.data)
        ytrain_true=dis(data[0])
        loss1=0.5*F.mean((ytrain_false)**2)
        loss2=0.5*F.mean((ytrain_true-1.0)**2)
        #loss1=0.5*F.mean(F.softplus(ytrain_false))
        #loss2=0.5*F.mean(F.softplus(-ytrain_true))
        loss_train_dis=loss1+loss2
        loss_train_dis.backward()
        optdis.update()
    #result[0].append(cuda.to_cpu(loss_train_gen.data))
    result[0].append(loss_train_gen.data)
    result[1].append(loss1.data)
    result[2].append(loss2.data)
    

In [22]:
foldername="output_GAN"
nepoch=1000
batch_size=10
Nfig=3
train_iter=siter(train,batch_size)
with tqdm(total=nepoch) as pbar:
    while train_iter.epoch < nepoch:
        pbar.update(train_iter.is_new_epoch)
        batch=train_iter.next()
        xtrain,ttrain=con.concat_examples(batch)
        ztrain=np.random.randn(len(xtrain)*zsize).reshape(len(xtrain),zsize).astype(np.float32)
        #ztest=cuda.to_gpu(np.random.randn(Nfig*zsize).reshape(Nfig,zsize).astype(np.float32))
        ztest=np.random.randn(Nfig*zsize).reshape(Nfig,zsize).astype(np.float32)
        #data=cuda.to_gpu([xtrain,ztrain])
        data=[xtrain,ztrain]
        learning_GAN(gen,dis,optgen,optdis,data,result,T=5)
        if train_iter.is_new_epoch==1 and train_iter.epoch%100==0:
            #display.clear_dropout(wait=True)
            #ohm.temp_image(train_iter.epoch,foldername+"/test",data[0],cuda_to_gpu(ztrain),ztest,gen,dis)
            ohm.temp_image(train_iter.epoch,foldername+"/test",data[0],ztrain,ztest,gen,dis)
            ohm.plot_result(result[0],"loss function of gen","step","loss function",0.0,0.6)
            ohm.plot_result(result[1],result[2],"loss function of dis","step","loss function",0.0,0.6)
save_model(dis,"test_dis.h5")
save_model(gen,"test_gen.h5")
        

  0%|          | 4/1000 [05:04<14:02:46, 50.77s/it]


KeyboardInterrupt: 

In [24]:
all_list=[]
ohm.add_labeled_data("princess",0,all_list)
ohm.add_labeled_data("white",1,all_list)
from ipywidgets import interact
import chainer

def GANview(z1=0.0,z2=0.0):
    zset=np.array([[z1,z2]]).astype(np.float32)
    with chainer.using_config("train",False),chainer.using_config("enable_backprop",False):
        temp=cuda.to_cpu(gen(cuda.to_gpu(zset)).data)
    plt.imshow(temp[0].transpose(1,2,0))
    plt.title("z1={},z2={}".format(z1,z2))
    plt.axis("off")

interact(GANview,z1=(-1.0,1.0,0.001),z2=(-1.0,1.0,0.001))


interactive(children=(FloatSlider(value=0.0, description='z1', max=1.0, min=-1.0, step=0.001), FloatSlider(val…

<function __main__.GANview(z1=0.0, z2=0.0)>

In [25]:
import numpy as np
import matplotlib.pyplot as plt

import chainer
import chainer.optimizers as Opt
import chainer.functions as F
import chainer.links as L
import chainer.datasets as ds
import chainer.dataset.convert as con
from chainer.iterators import SerialIterator as siter
from chainer import optimizer_hooks as oph
from chainer import Variable,Chain,config,cuda

from tqdm import tqdm
from IPython import display
import princess as ohm

In [31]:
all_list=[]
ohm.add_labeled_data("princess",0,all_list)
dataset=ds.LabeledImageDataset(all_list)
dataset=ds.TransformDataset(dataset,ohm.labeled64)
D=len(dataset)
trainA,testA=ds.split_dataset_random(dataset,D//2)

all_list=[]
ohm.add_labeled_data("white",0,all_list)
dataset=ds.LabeledImageDataset(all_list)
dataset=ds.TransformDataset(dataset,ohm.labeled64)
D=len(dataset)
trainB,testB=ds.split_dataset_random(dataset,D//2)

xtrainA,_=con.concat_examples(trainA)
xtestA,_=con.concat_examples(testA)
xtrainB,_=con.concat_examples(trainB)
xtestB,_=con.concat_examples(testB)
DtrainA,ch,Ny,Nx=xtrainA.shape
DtestA=len(xtestA)
print(DtrainA,DtestA,ch,Ny,Nx)
DtrainB,ch,Ny,Nx=xtrainB.shape
DtestB=len(xtestB)
print(DtrainB,DtestB,ch,Ny,Nx)


100 100 3 64 64
100 100 3 64 64


In [32]:
class Generator(Chain):
    def __init__(self,ch=ch,H1=64,H2=128,H3=256,H4=512):
        layers={}
        layers["cnn1"]=ohm.CBR(ch,H1,"down")
        layers["cnn2"]=ohm.CBR(H1,H2,"down")
        layers["cnn3"]=ohm.CBR(H2,H3,"down")
        layers["cnn4"]=ohm.CBR(H3,H4,"down")
        layers["l1"]=L.Linear(H4*4*4,H4*4*4)
        layers["bnorm1"]=L.BatchNormalization(H4*4*4)
        layers["dcnn1"]=ohm.CBR(H4,H3,"up")
        layers["dcnn2"]=ohm.CBR(H3,H2,"up")
        layers["dcnn3"]=ohm.CBR(H2,H1,"up")
        layers["dcnn4"]=ohm.CBR(H1,ch,"up")
        super().__init__(**layers)
    def __call__(self,x):
        h=self.cnn1(x)
        h=self.cnn2(h)
        h=self.cnn3(h)
        h=self.cnn4(h)
        h=self.l1(h)
        h=self.bnorm1(h)
        h=F.relu(h)
        h=h.reshape(len(h),512,4,4)
        h=self.dcnn1(h)
        h=self.dcnn2(h)
        h=self.dcnn3(h)
        h=self.dcnn4(h)
        y=F.clip(h,0.0,1.0)
        return y

class Discriminator(Chain):
    def __init__(self,C,ch=ch,H1=64,H2=128,H3=256,H4=512):
        layers={}
        layers["cnn1"]=ohm.CBR(ch,H1,"down",bn=False,act=F.leaky_relu)
        layers["cnn2"]=ohm.CBR(H1,H2,"down",bn=False,act=F.leaky_relu)
        layers["cnn3"]=ohm.CBR(H2,H3,"down",bn=False,act=F.leaky_relu)
        layers["cnn4"]=ohm.CBR(H3,H4,"down",bn=False,act=F.leaky_relu)
        layers["l1"]=L.Linear(None,C)
        super().__init__(**layers)
    def __call__(self,x):
        h=self.cnn1(x)
        h=self.cnn2(h)
        h=self.cnn3(h)
        h=self.cnn4(h)
        h=self.l1(h)
        y=F.dropout(h)
        return y    

In [35]:
gen_AtoB=Generator()
gen_BtoA=Generator()

C=1
dis_A=Discriminator(C)
dis_B=Discriminator(C)
optgen_AtoB=Opt.Adam(alpha=0.0005,beta1=0.5)
optgen_AtoB.setup(gen_AtoB)
optgen_BtoA=Opt.Adam(alpha=0.0005,beta1=0.5)
optgen_BtoA.setup(gen_BtoA)
optdis_A=Opt.Adam(alpha=0.0001,beta1=0.5)
optdis_A.setup(dis_A)
optdis_B=Opt.Adam(alpha=0.0001,beta1=0.5)
optdis_B.setup(dis_B)
cuda.get_device(0).use()
gen_AtoB.to_gpu()
gen_BtoA.to_gpu()
dis_A.to_gpu()
dis_B.to_gpu()
train_gen_loss_A=[]
train_dis_loss_A1=[]
train_dis_loss_A2=[]
train_gen_loss_B=[]
train_dis_loss_B1=[]
train_dis_loss_B2=[]
resultA=[train_gen_loss_A,train_dis_loss_A1,train_dis_loss_A2]
resultB=[train_gen_loss_B,train_dis_loss_B1,train_dis_loss_B2]

RuntimeError: CUDA environment is not correctly set up
(see https://github.com/chainer/chainer#installation).CuPy is not correctly installed.

If you are using wheel distribution (cupy-cudaXX), make sure that the version of CuPy you installed matches with the version of CUDA on your host.
Also, confirm that only one CuPy package is installed:
  $ pip freeze

If you are building CuPy from source, please check your environment, uninstall CuPy and reinstall it with:
  $ pip install cupy --no-cache-dir -vvvv

Check the Installation Guide for details:
  https://docs-cupy.chainer.org/en/latest/install.html

original error: libcuda.so.1: cannot open shared object file: No such file or directory

In [36]:
def learning_consist(gen_BtoA,gen_AtoB,optgen_BtoA,optgen_AtoB,data,T=5):
    a=10
    for time in range(T):
        optgen_BtoA.target.cleargrads()
        optgen_AtoB.target.cleargrads()
        ytemp1=gen_BtoA(data[1])
        ytemp2=gen_AtoB(data[0])
        loss_train=0.5*a*F.mean_absolute_error(ytemp1,data[1])+0.5*a*F.mean_absolute_error(ytemp2,data[0])
        loss_train.backward()
        result=loss_train.data
        optgen_BtoA.update()
        optgen_AtoB.update()
        
        

In [37]:
def learning_L1(gen_BtoA,gen_AtoB,optgen_BtoA,optgen_AtoB,data,T=5):
    a=10
    for time in range(T):
        optgen_BtoA.target.cleargrads()
        optgen_AtoB.target.cleargrads()
        ytemp1=gen_BtoA(data[0])
        ytrain1=gen_AtoB(ytemp1)
        ytemp2=gen_AtoB(data[1])
        ytrain2=gen_BtoA(ytemp2)
        loss_train=0.5*a*F.mean_absolute_error(ytrain1,data[0])+0.5*a*F.mean_absolute_error(ytrain2,data[1])
        loss_train.backward()
        result=loss_train.data
        optgen_BtoA.update()
        optgen_AtoB.update()

In [38]:
def temp_image2(epoch,filename,dataA,dataB,gen_AtoB,gen_BtoA,dis_A,dis_B):
    print("epoch",epoch)
    with chainer.using_config("train",False),chainer.using_config("enable_backprop",False):
        xtestAB=gen_AtoB(cuda.to_gpu(dataA))
        scoreAB=dis_B(xtestAB)
        xtestABA=gen_BtoA(xtestAB)
        xtestBA=gen_BtoA(cuda.to_gpu(dataB))
        scoreBA=dis_A(xtestBA)
        xtestBAB=gen_AtoB(xtestBA)
    kA=np.random.randint(len(dataA))
    kB=np.random.randint(len(dataB))
    plt.figure(figsize=(12,9))
    plt.subplot(3,2,1)
    plt.axis("off")
    plt.title("imageA")
    plt.imshow(dataA[kA,:,:,:].transpose(1,2,0))
    plt.subplot(3,2,2)
    plt.axis("off")
    plt.imshow(dataB[kB,:,:,:].transpose(1,2,0))
    plt.axis("off")
    plt.title("imageB")
    plt.subplot(3,2,3)
    plt.axis("off")
    plt.title("{}".format(cuda.to_cpu(scoreAB[kA].data)))
    plt.imshow(cuda.to_cpu(xtestAB[kA,:,:,:].data).transpose(1,2,0))
    plt.subplot(3,2,4)
    plt.axis("off")
    plt.title("{}".format(cuda.to_cpu(scoreBA[kB].data)))
    plt.imshow(cuda.to_cpu(xtestBA[kB,:,:,:].data).transpose(1,2,0))
    plt.subplot(3,2,5)
    plt.axis("off")
    plt.title("A to B to A")
    plt.imshow(cuda.to_cpu(xtestABA[kA:,:,:].data).transpose(1,2,0))
    plt.subplot(3,2,6)
    plt.axis("off")
    plt.title("B to A to B")
    plt.imshow(cuda.to_cpu(xtestBAB[kB,:,:,:].data).transpose(1,2,0))
    plt.savefig(filename+"_{0:03d}.png".format(epoch))
    plt.show()

In [ ]:
output_folder="output_princess"
nepoch=3000
batch_size=10
train_iter_A=siter(trainA,batch_size)
train_iter_B=siter(trainB,batch_size)
with tqdm(total=nepoch) as pbar:
    while train_iter_A.epoch<nepoch:
        pbar.update(train_iter_A.is_new_epoch)
        batchA=train_iter_A.next()
        batchB=train_iter_B.next()
        xtrainA,_=con.concat_examples(batchA)
        xtrainB,_=con.concat_examples(batchB)
        dataBA=cuda.to_gpu([xtrainB,xtrainA])
        ohm.learning_L1(gen_BtoA,gen_AtoB,optgen_BtoA,optgen_AtoB,dataBA)
        ohm.learning_consist(gen_BtoA,gen_AtoB,optgen_BtoA,optgen_AtoB,dataBA)
        ohm.learning_GAN(gen_AtoB,dis_B,optgen_AtoB,optdis_B,dataBA,resultB,T=5)
        dataAB=cuda.to_gpu([xtrainA,xtrainB])
        ohm.learning_GAN(gen_BtoA,dis_A,optgen_BtoA,optdis_A,dataAB,resultA,T=5)
        if train_iter_A.epoch%100==0:
            ohm.temp_image2(train_iter_A.epoch,output_folder+"/test",xtestA,xtestB,gen_AtoB,gen_BtoA,dis_A,dis_B)
            ohm.plot_result(resultA[0],"loss function A to B of gen in training","step","loss function",0.0,0.6)
            ohm.plot_result2(resultA[1],resultA[2],"loss function A to B of dis","step","loss function",0.0,0.6)
            ohm.plot_result(resultB[0],"loss function B to A of gen in training","step","loss function",0.0,0.6)
            ohm.plot_result2(resultB[1],resultB[2],"loss function B to A of dis","step","loss function",0.0,0.6)
save_model(dis,"test_dis.h6")
save_model(gen,"test_gen.h6")
        